Importing pandas ; Importing the final progress dataframe ; Importing seaborn ; Importing numpy ; Importing the average closing price ; Importing the price csv ; Setting price csv index ; Preparing the final dataframe for prediction

In [235]:
import pandas as pd
import seaborn as sns
import numpy as np

df_closing_prediction=pd.read_csv('../../data/output/price-prediction-by-book-value-growth.csv',sep=',')
df_closing_prediction=df_closing_prediction.set_index('Year')[['3 Prediction']]

df_price=pd.read_csv('../../data/input/price.csv',sep=',')
df_price['Date']=pd.to_datetime(df_price['Date'])
df_price['Year']=df_price['Date'].dt.year
df_price['Month']=df_price['Date'].dt.month
df_price=df_price.set_index(['Year','Month']).drop(columns=['Date','Adj Close','Volume'])

df_normal=pd.read_csv('../../data/output/monthly-progress-by-book-value/mean/AR.csv',sep=';')[['Year','Month','AR 5 Mean']]
df_normal=df_normal.set_index(['Year','Month'])

df_normal=pd.merge(df_price,df_normal,how='left',left_index=True,right_index=True)
df_normal=pd.merge(df_normal,df_closing_prediction,how='left',left_index=True,right_index=True)
df_normal=df_normal.dropna()
days=len(df_normal)

Preparing three dataframe for mean prediction tolerance

In [236]:
tolerance_mean=0.029

df_lower,df_upper=df_normal.copy(),df_normal.copy()

df_normal['Estimated Price']=df_normal['AR 5 Mean']*df_normal['3 Prediction']
df_lower['Estimated Price']=(df_lower['AR 5 Mean']-tolerance_mean)*df_lower['3 Prediction']
df_upper['Estimated Price']=(df_upper['AR 5 Mean']+tolerance_mean)*df_upper['3 Prediction']

Prompting investor buy deviation from AR mean and profit margin

In [237]:
# the buy and sell deviation
deviation_buy=0.01
margin_profit=0.014

# security tax
security_tax=0.004

Generating buy price and sell price columns

In [238]:
df_normal['Buy Price']=df_normal['Estimated Price']*(1-deviation_buy)
df_normal['Sell Price']=df_normal['Buy Price']*(1+margin_profit)

df_lower['Buy Price']=df_lower['Estimated Price']*(1-deviation_buy)
df_lower['Sell Price']=df_lower['Buy Price']*(1+margin_profit)

df_upper['Buy Price']=df_upper['Estimated Price']*(1-deviation_buy)
df_upper['Sell Price']=df_upper['Buy Price']*(1+margin_profit)

df_upper=df_upper.drop(columns=['3 Prediction','AR 5 Mean','Estimated Price'])
df_lower=df_lower.drop(columns=['3 Prediction','AR 5 Mean','Estimated Price'])
df_normal=df_normal.drop(columns=['3 Prediction','AR 5 Mean','Estimated Price'])

print(df_normal.head())
print(df_lower.head())
print(df_upper.head())

              Open    High     Low   Close  Buy Price  Sell Price
Year Month                                                       
2017 1      3100.0  3100.0  3100.0  3100.0   3039.075  3081.62205
     1      3100.0  3155.0  3045.0  3155.0   3039.075  3081.62205
     1      3170.0  3175.0  3130.0  3140.0   3039.075  3081.62205
     1      3140.0  3155.0  3125.0  3135.0   3039.075  3081.62205
     1      3090.0  3140.0  3090.0  3120.0   3039.075  3081.62205
              Open    High     Low   Close    Buy Price   Sell Price
Year Month                                                          
2017 1      3100.0  3100.0  3100.0  3100.0  2931.502667  2972.543704
     1      3100.0  3155.0  3045.0  3155.0  2931.502667  2972.543704
     1      3170.0  3175.0  3130.0  3140.0  2931.502667  2972.543704
     1      3140.0  3155.0  3125.0  3135.0  2931.502667  2972.543704
     1      3090.0  3140.0  3090.0  3120.0  2931.502667  2972.543704
              Open    High     Low   Close    Buy Price

Defining profitability apply function

In [239]:
hold_stock=False

def determine_takeProfit(row):
    
    global hold_stock
    
    if hold_stock:
        if row['Sell Price']<=row['High']:
            hold_stock=False
            return True
    else:
        if row['Buy Price']>=row['Low']:
            hold_stock=True
    
    return False
    

Applying profitability function to final dataframe

In [240]:
df_normal['Take Profit']=df_normal.apply(determine_takeProfit,axis=1)
hold_stock=False
df_lower['Take Profit']=df_lower.apply(determine_takeProfit,axis=1)
hold_stock=False
df_upper['Take Profit']=df_upper.apply(determine_takeProfit,axis=1)

df_normal=df_normal.groupby(['Year','Month']).sum()['Take Profit']
df_lower=df_lower.groupby(['Year','Month']).sum()['Take Profit']
df_upper=df_upper.groupby(['Year','Month']).sum()['Take Profit']

Fill the final dataframe with the most profitable month

In [241]:
df_final=pd.DataFrame(index=df_normal.index)

years=df_final.index.get_level_values('Year')

for year in years:
    months=df_final.loc[year].index.get_level_values('Month')
    for month in months:
        df_final.loc[(year,month),'Take Profit Day']=max(df_normal.loc[(year,month)],df_lower.loc[(year,month)],df_upper.loc[(year,month)])

print('With adding tolerance mean and a profit margin of',(margin_profit-security_tax)*100,'%, there is',df_final['Take Profit Day'].sum(),'profit day of',days,' days')

With adding tolerance mean and a profit margin of 1.0 %, there is 145.0 profit day of 1789  days
